COLAB DI GAIA BIANCO, ALESSANDRO CORTI, ROBERTA MIRAGLIA E ANTONIETTA TODISCO

In [ ]:
!pip install torch transformers sentence-transformers scikit-learn pandas numpy tqdm rank_bm25 pyarrow -q

In [ ]:
import os
import zipfile
import urllib.request

DATASET_URL = "https://github.com/fabsilvestri/aiml_hackathon_data/releases/download/v1.0/kaggle_data.zip"
DATA_DIR = "data"

print("Downloading dataset...")
urllib.request.urlretrieve(DATASET_URL, "kaggle_data.zip")
os.makedirs(DATA_DIR, exist_ok=True)
with zipfile.ZipFile("kaggle_data.zip", "r") as zf:
    zf.extractall(DATA_DIR)
os.remove("kaggle_data.zip")
print("Dataset ready!")

Dataset ready!


In [ ]:
import pandas as pd
import random

test_queries = pd.read_csv(f"{DATA_DIR}/test.csv")
collection = pd.read_parquet(f"{DATA_DIR}/collection.parquet")
ts = pd.read_parquet(f"{DATA_DIR}/train.parquet")
vs = pd.read_parquet(f"{DATA_DIR}/val.parquet")
all_pids = collection["pid"].astype(str).tolist()

print(f"Loaded {len(test_queries)} test queries and {len(all_pids)} passages.")

Loaded 4912 test queries and 651625 passages.


Librerie utili:

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import sentence_transformers
import torch
import numpy
import rank_bm25

Visualizziamo i dataframe:

In [ ]:
test_queries.head()

,id,query
0,575616,what are the weight an ankole cattle
1,1057708,confirmation number definition
2,1102262,definition of sadducees
3,1083157,what does qod stand for in medical terms
4,26485,argue the meaning of a debate


In [ ]:
collection.head()

,pid,passage
0,8230693,Chris Gill of Guitar World commented: Stevie R...
1,427355,"1 However, if your baby is consistently breaki..."
2,8528367,Letter of Indemnity. Personal Affidavit and In...
3,4028334,"Introducing Microsoft Teams, your new chat-bas..."
4,6753791,By Maria Korologou -. Many arts and sciences w...


In [ ]:
ts.head()

,qid,query,pid,label
0,948654,when does water boil in celsius,4150796,1
1,948654,when does water boil in celsius,2834350,0
2,948654,when does water boil in celsius,5340956,0
3,948654,when does water boil in celsius,2270491,0
4,737016,what is cyber forensics,1185025,1


Trasformiamo i dataframe in liste con una funzione:

In [ ]:
def df2list(df):
  return df.values.tolist()

Preparazione dati:

In [ ]:
tqlist = df2list(test_queries)
clist = df2list(collection)
tqdict = {tqlist[i][0] : tqlist[i][1] for i in range(len(tqlist))}
cdict = {clist[i][0] : clist[i][1] for i in range(len(clist))}
qlist = df2list(test_queries['query'])
plist = df2list(collection['passage'])
qidslist = df2list(test_queries["id"])
pidslist = df2list(collection["pid"])

Funzioni che fanno il ranking:

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from tqdm import tqdm

# --- CONFIGURAZIONE ---
# Sostituisci con i tuoi dati reali
queries = qlist  # Simula 4912 query
passaggi = plist # Simula 65000 passaggi

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilizzando: {device}")

# --- 1. CARICAMENTO MODELLI ---
# Bi-Encoder per il retrieval veloce
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)
# Cross-Encoder per il ranking di alta qualità
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)

# --- 2. EMBEDDING DEI PASSAGGI ---
print("Generazione embeddings per i passaggi...")
corpus_embeddings = bi_encoder.encode(
    passaggi,
    batch_size=64,
    show_progress_bar=True,
    convert_to_tensor=True
)

# --- 3. RETRIEVAL E RERANKING ---
results = []

print("Elaborazione query...")
for query in tqdm(queries):
    # FASE A: Bi-Encoder Retrieval (Top 100)
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=100)[0]

    # Prepara i dati per il Cross-Encoder
    passage_indices = [hit['corpus_id'] for hit in hits]
    cross_inputs = [[query, passaggi[idx]] for idx in passage_indices]

    # FASE B: Cross-Encoder Reranking
    cross_scores = cross_encoder.predict(cross_inputs, batch_size=128, show_progress_bar=False)

    # Combina e ordina
    for i in range(len(hits)):
        hits[i]['score'] = cross_scores[i]
        hits[i]['text'] = passaggi[hits[i]['corpus_id']]

    # Prendi i Top 10 definitivi
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)[:10]
    results.append({"query": query, "top_10": hits})

# --- 4. SALVATAGGIO ---
# Esempio di visualizzazione del primo risultato
print("\nTop 1 Risultato per la prima query:")
print(f"Query: {results[0]['query']}")
print(f"Passaggio: {results[0]['top_10'][0]['text']} (Score: {results[0]['top_10'][0]['score']:.4f})")

Utilizzando: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Generazione embeddings per i passaggi...


Batches:   0%|          | 0/10182 [00:00<?, ?it/s]

Elaborazione query...


100%|██████████| 4912/4912 [17:26<00:00,  4.69it/s]


Top 1 Risultato per la prima query:
Query: what are the weight an ankole cattle
Passaggio: Lyre and circular shapes are preferable to flat. The Ankole-Watusi is medium in size, with cows weighing 900 - 1200 pounds and bulls weighing 1000 - 1600 pounds. Newborn calves weigh 30 - 50 pounds. This small birth-weight makes Ankole-Watusi bulls useful for breeding to first-calf-heifers of other breeds. During the day, calves sleep together, with an auntie cow for protection. At night, the herd-members sleep together, with the calves in the center of the group for protection. (Score: 9.4026)


In [ ]:
result_ids = []
for d in results:
  result_ids.append([d["query"], [d["top_10"][i]['text'] for i in range(10)]])
result_ids[2]
out = []
tqdict_inv = {v: k for k, v in tqdict.items()}
cdict_inv = {v: k for k, v in cdict.items()}
for q, ps in result_ids:
  x = []
  for p in ps:
    x.append(cdict_inv[p])
  out.append([tqdict_inv[q], ' '.join(x)])
out[1]


[1057708,
 '7172579 7172572 7655754 5126803 1747414 643935 1228282 592884 7899014 1761448']

In [ ]:
# --- 4. SALVATAGGIO FILE ---
results_df = pd.DataFrame(out, columns = ['id', 'expected'])

# Controlli rapidi
print(f"\nRighe generate: {len(results_df)}")
print("Esempio prime righe:")
print(results_df.head())

# Salvataggio su disco
results_df.to_csv("submission.csv", index=False)
print("File 'submission.csv' salvato correttamente.")


Righe generate: 4912
Esempio prime righe:
        id                                           expected
0   575616  7416273 3701827 7563586 8395099 2942337 294233...
1  1057708  7172579 7172572 7655754 5126803 1747414 643935...
2  1102262  7129543 7129542 7129541 522934 3162238 490455 ...
3  1083157  7140047 873366 964971 964970 964974 3374012 71...
4    26485  7983408 2347250 766817 478294 1637957 1987744 ...
File 'submission.csv' salvato correttamente.
